In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local[*]").getOrCreate()

In [3]:
from pyspark.sql import functions as f
from pyspark.sql import Window

In [4]:
data = sparkSession.read.parquet("/data/sample264")

In [5]:
data.take(10)

[Row(userId=13065, trackId=944906, artistId=978428, timestamp=1501588527),
 Row(userId=101897, trackId=799685, artistId=989262, timestamp=1501555608),
 Row(userId=215049, trackId=871513, artistId=988199, timestamp=1501604269),
 Row(userId=309769, trackId=857670, artistId=987809, timestamp=1501540265),
 Row(userId=397833, trackId=903510, artistId=994595, timestamp=1501597615),
 Row(userId=501769, trackId=818149, artistId=994975, timestamp=1501577955),
 Row(userId=601353, trackId=958990, artistId=973098, timestamp=1501602467),
 Row(userId=710921, trackId=916226, artistId=972031, timestamp=1501611582),
 Row(userId=6743, trackId=801006, artistId=994339, timestamp=1501584964),
 Row(userId=152407, trackId=913509, artistId=994334, timestamp=1501571055)]

In [6]:
userArtist = data.groupBy('userId', 'artistId').count()

In [7]:
userArtist.take(10)

[Row(userId=484714, artistId=1000564, count=2),
 Row(userId=685378, artistId=974357, count=8),
 Row(userId=531701, artistId=969480, count=5),
 Row(userId=341232, artistId=977291, count=1),
 Row(userId=554281, artistId=985827, count=1),
 Row(userId=395708, artistId=975337, count=1),
 Row(userId=646244, artistId=1001300, count=2),
 Row(userId=108592, artistId=991179, count=1),
 Row(userId=245658, artistId=997265, count=3),
 Row(userId=485786, artistId=993060, count=20)]

In [8]:
def norm(df, key1, key2, field, n): 
    
    window = Window.partitionBy(key1).orderBy(f.col(field).desc())
        
    topsDF = df.withColumn('row_number', f.row_number().over(window)) \
        .filter(f.col('row_number') <= n) \
        .drop(f.col('row_number')) 
        
    tmpDF = topsDF.groupBy(f.col(key1)).agg(f.col(key1), f.sum(f.col(field)).alias('sum_' + field))
   
    normalizedDF = topsDF.join(tmpDF, key1, 'inner') \
        .withColumn('norm_' + field, f.col(field) / f.col('sum_' + field)) \
        .cache()

    return normalizedDF

In [9]:
userArtistNorm = norm(userArtist, 'userId', 'artistId', 'count', 100) \
        .select('userId', 'artistId', 'norm_count')

In [10]:
userArtistNorm.take(30)

[Row(userId=3175, artistId=981306, norm_count=0.2222222222222222),
 Row(userId=3175, artistId=995274, norm_count=0.1111111111111111),
 Row(userId=3175, artistId=986492, norm_count=0.1111111111111111),
 Row(userId=3175, artistId=976051, norm_count=0.1111111111111111),
 Row(userId=3175, artistId=1000709, norm_count=0.1111111111111111),
 Row(userId=3175, artistId=984798, norm_count=0.1111111111111111),
 Row(userId=3175, artistId=969751, norm_count=0.1111111111111111),
 Row(userId=3175, artistId=1000564, norm_count=0.1111111111111111),
 Row(userId=5518, artistId=978963, norm_count=0.5),
 Row(userId=5518, artistId=984128, norm_count=0.3333333333333333),
 Row(userId=5518, artistId=969429, norm_count=0.16666666666666666),
 Row(userId=5803, artistId=982335, norm_count=1.0),
 Row(userId=6654, artistId=1002715, norm_count=0.2),
 Row(userId=6654, artistId=985758, norm_count=0.2),
 Row(userId=6654, artistId=987351, norm_count=0.2),
 Row(userId=6654, artistId=987809, norm_count=0.2),
 Row(userId=66

In [11]:
window = Window.orderBy(f.col('norm_count').desc())
    
userArtistList = userArtistNorm.withColumn('position', f.rank().over(window)) \
    .filter(f.col('position') < 40) \
    .orderBy('userId', 'artistId') \
    .select('userId', 'artistId') \
    .take(40)

In [12]:
for val in userArtistList:
    print "%s %s" % val

66 993426
116 974937
128 1003021
131 983068
195 997265
215 991696
235 990642
288 1000564
300 1003362
321 986172
328 967986
333 1000416
346 982037
356 974846
374 1003167
428 993161
431 969340
445 970387
488 970525
542 969751
612 987351
617 970240
649 973851
658 973232
662 975279
698 995788
708 968848
746 972032
747 972032
776 997265
784 969853
806 995126
811 996436
837 989262
901 988199
923 977066
934 990860
957 991171
989 975339
999 968823
